In [37]:
import pandas as pd
import seaborn as sns
import scipy

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

import pprint
pp = pprint.PrettyPrinter(indent=4)

print("Using pandas %s version" % pd.__version__)
print("Using seaborn %s version"% sns.__version__)
print("Using scipy %s version" % scipy.__version__)


Using pandas 1.1.3 version
Using seaborn 0.11.0 version
Using scipy 1.5.2 version


In [38]:
ndcg = pd.read_table(r'C:\Projects\RecSys2020\results\figure3.txt',sep='\t',header=4,nrows=13)

#print(ndcg)
ndcg.head()

,Target size,iMF (full),iMF (test),kNN (full/test),Normalized kNN (full),Normalized kNN (test),Average Rating,Popularity,Random
0,0,0.839327,0.868961,0.839908,0.813941,0.855383,0.846539,0.793741,0.691696
1,1,0.836227,0.864861,0.837244,0.800879,0.843466,0.829542,0.787801,0.658244
2,2,0.833667,0.861136,0.834542,0.789320,0.832446,0.814018,0.782174,0.629268
3,5,0.825869,0.848994,0.827059,0.758801,0.801115,0.770456,0.765180,0.551586
4,10,0.815657,0.831748,0.815841,0.720109,0.757919,0.710572,0.741482,0.452840


In [39]:
precision = pd.read_table(r'C:\Projects\RecSys2020\results\figure3.txt',sep='\t',header=19,nrows=13)

#print(precision)

precision.head()

,Target size,iMF (full),iMF (test),kNN (full/test),Normalized kNN (full),Normalized kNN (test),Average Rating,Popularity,Random
0,0,0.660162,0.684576,0.659460,0.642364,0.671166,0.665384,0.624252,0.549192
1,1,0.658219,0.682199,0.657715,0.634917,0.664589,0.656149,0.620825,0.532238
2,2,0.656775,0.679960,0.656050,0.627334,0.658281,0.647629,0.617344,0.514189
3,5,0.651331,0.672437,0.650490,0.604705,0.639093,0.620874,0.606199,0.461868
4,10,0.643854,0.661268,0.642606,0.574901,0.610156,0.580467,0.589563,0.388930


In [40]:
recall = pd.read_table(r'C:\Projects\RecSys2020\results\figure3.txt',sep='\t',header=34,nrows=13)

#print(recall)
recall.head()

,Target size,iMF (full),iMF (test),kNN (full/test),Normalized kNN (full),Normalized kNN (test),Average Rating,Popularity,Random
0,0,0.618379,0.634086,0.619114,0.604247,0.628770,0.625773,0.602127,0.564606
1,1,0.616380,0.631863,0.617081,0.595366,0.622863,0.617662,0.598392,0.548670
2,2,0.614040,0.629787,0.615118,0.585870,0.617216,0.610010,0.594605,0.531592
3,5,0.605853,0.621514,0.608245,0.552917,0.598152,0.583340,0.581139,0.471232
4,10,0.592157,0.608500,0.596437,0.496597,0.564844,0.535832,0.558513,0.366818


In [56]:
for i in recall.columns[1:]:
    print(i, recall[i].rank(axis=1,ascending=False))

ValueError: No axis named 1 for object type Series

In [100]:
for i,j in recall.groupby(['Target size']):
    #print(type(i))
    #print(type(j))
    #print(j['iMF (full)'])
    #print(j.columns[1:])
    print(j.rank())
    


   Target size  iMF (full)  iMF (test)  kNN (full/test)  \
0          1.0         1.0         1.0              1.0   

   Normalized kNN (full)  Normalized kNN (test)  Average Rating  Popularity  \
0                    1.0                    1.0             1.0         1.0   

   Random  
0     1.0  
   Target size  iMF (full)  iMF (test)  kNN (full/test)  \
1          1.0         1.0         1.0              1.0   

   Normalized kNN (full)  Normalized kNN (test)  Average Rating  Popularity  \
1                    1.0                    1.0             1.0         1.0   

   Random  
1     1.0  
   Target size  iMF (full)  iMF (test)  kNN (full/test)  \
2          1.0         1.0         1.0              1.0   

   Normalized kNN (full)  Normalized kNN (test)  Average Rating  Popularity  \
2                    1.0                    1.0             1.0         1.0   

   Random  
2     1.0  
   Target size  iMF (full)  iMF (test)  kNN (full/test)  \
3          1.0         1.0         

In [127]:
ndcg.head()
pp.pprint(ndcg.groupby(['Target size']).length)


AttributeError: 'DataFrameGroupBy' object has no attribute 'length'

In [242]:
artists = []
xasix = []
colors = ('red','blue','green','magenta', 'black', 'purple', 'grey', 'orange')
ndcg.set_index('Target size') 
for i,j in ndcg.groupby(['Target size'], axis=0):
    del j['Target size']
    w = j.rank(axis=1,ascending=False)
    q = np.array(w)[0]
    print(q, i)
    artists.append(q)
    xasix.append(i)
    
z = np.array(artists)
rrr = []
for i,t in enumerate(ndcg.keys()[1:]):
    #print(i,t)
    #print(z[i][0])
    print(z[:,i],t)
    rrr.append([t].extend(z[:,i]))
    
#z.extend([artists[i][1:2] for i in range(len(artists))])
#pp.pprint(np.concatenate(z))
pp.pprint(rrr)
        

[5. 1. 4. 6. 2. 3. 7. 8.] 0
[4. 1. 3. 6. 2. 5. 7. 8.] 1
[3. 1. 2. 6. 4. 5. 7. 8.] 2
[3. 1. 2. 7. 4. 5. 6. 8.] 5
[3. 1. 2. 6. 4. 7. 5. 8.] 10
[1. 2. 3. 6. 5. 7. 4. 8.] 20
[1. 3. 2. 4. 6. 7. 5. 8.] 50
[1. 3. 2. 4. 6. 7. 5. 8.] 100
[1. 3. 2. 4. 6. 7. 5. 8.] 200
[1. 4. 2. 3. 6. 7. 5. 8.] 500
[1. 5. 2. 3. 6. 7. 4. 8.] 1000
[1. 5. 2. 3. 6. 7. 4. 8.] 2000
[1. 5. 2. 4. 6. 7. 3. 8.] 3706
[5. 4. 3. 3. 3. 1. 1. 1. 1. 1. 1. 1. 1.] iMF (full)
[1. 1. 1. 1. 1. 2. 3. 3. 3. 4. 5. 5. 5.] iMF (test)
[4. 3. 2. 2. 2. 3. 2. 2. 2. 2. 2. 2. 2.] kNN (full/test)
[6. 6. 6. 7. 6. 6. 4. 4. 4. 3. 3. 3. 4.] Normalized kNN (full)
[2. 2. 4. 4. 4. 5. 6. 6. 6. 6. 6. 6. 6.] Normalized kNN (test)
[3. 5. 5. 5. 7. 7. 7. 7. 7. 7. 7. 7. 7.] Average Rating
[7. 7. 7. 6. 5. 4. 5. 5. 5. 5. 4. 4. 3.] Popularity
[8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.] Random
[None, None, None, None, None, None, None, None]


In [232]:
pp.pprint(ndcg.keys()[1:])

Index(['iMF (full)', 'iMF (test)', 'kNN (full/test)', 'Normalized kNN (full)',
       'Normalized kNN (test)', 'Average Rating', 'Popularity', 'Random'],
      dtype='object')


In [213]:
m = np.array(artists)
m[:,0]

array([5., 4., 3., 3., 3., 1., 1., 1., 1., 1., 1., 1., 1.])

In [175]:
def figure3(data, legend=False, label=False, title=''):
    
    
    result = [list(a) for a in zip(data['Target size'], 
                                   data.Full.rank(ascending=False),
                                   data.Unbiased.rank(ascending=False),
                                   data.Test.rank(ascending=False))]

    artists = []
    for row, color in zip(result, ('red','blue','green','magenta', 'black', 'purple', 'grey', 'orange')):
        artists.append(plt.Line2D(xdata=[1,2,3], ydata=[row[1:]], lw=1, color=color, marker='o'))

    fig, ax = plt.subplots()

    for artist in artists:
        a = ax.add_artist(artist)

    ax.set_ybound([0.8,8.2])
    ax.set_xbound([0.94,3.06])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    plt.gca().invert_yaxis()
    if (label):
        plt.ylabel('System ranking')
    if (legend):
        plt.legend(artists, recall.Recommender, bbox_to_anchor=(1.05, 1), loc='upper left', title='Recommender', title_fontsize='xx-large')
    #plt.tick_params(
    #    axis='x',          # changes apply to the x-axis
    #    which='both',      # both major and minor ticks are affected
    #    bottom=False,      # ticks along the bottom edge are off
    #    top=False,         # ticks along the top edge are off
    #    labelbottom=False) # labels along the bottom edge are off
    #plt.rcParams["figure.figsize"] = (2,6)
    plt.title(title)
    plt.xticks([1,2,3], ['Full', 'Unbiased', 'Test'])
    return result

In [176]:
r = figure3(recall, label=True, title='Recall@10')

AttributeError: 'DataFrame' object has no attribute 'Full'

In [ ]:
p = figure3(precision, title='Precision@10')

In [ ]:
n = figure3(ndcg, title='nDCG@10', legend=True)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharex='row', constrained_layout=True)
#fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, nrows=1, constrained_layout=False)

def combine_figure(data, ax):
    artists = []
    for row, color in zip(data, ('red','blue','green','magenta', 'black', 'purple', 'grey', 'orange')):
        artists.append(plt.Line2D(xdata=[1,2,3], ydata=[row[1:]], lw=1, color=color, marker='o'))

    for artist in artists:
        ax.add_artist(artist)

    ax.set_ybound([0.8,8.2])
    ax.set_xbound([0.94,3.06])
    #ax.spines['top'].set_visible(False)
    #ax.spines['right'].set_visible(False)
    #ax.spines['bottom'].set_visible(False)
    #ax.spines['left'].set_visible(False)
    return artists
    
combine_figure(p, ax1)
combine_figure(r, ax2)
artists = combine_figure(n, ax3)

plt.gca().invert_yaxis()

plt.xticks([1,2,3], ['Full', 'Unbiased', 'Test'])
    
plt.legend(artists, recall.Recommender, bbox_to_anchor=(1.05, 1), loc='upper left', title='Recommender', title_fontsize='x-large')
#plt.ylabel('System ranking')
fig.tight_layout(h_pad=5, w_pad=5)

#fig.set_constrained_layout_pads(w_pad=2/72, h_pad=2/72, hspace=0.2, wspace=0.2)
